<p style = "font-weight:bold; font-size:200%">PowerShell コマンド結果一括確認</p>

In [ ]:
$ErrorActionPreference = 'Continue'

# 確認

## 定義

In [ ]:
$definition = @(
    @{
        Name = 'Display Hostname'
        Command = 'hostname'
    }
    @{
        Name = 'Ping Loopback'
        Command = 'Test-Connection 127.0.0.1 -Quiet'
        Expect = $true
    }
)

## チェック

In [ ]:
$i = 1
$ngCount = 0; $errCount = 0
$definition | ForEach-Object {    
    $return = (Invoke-Expression $_.Command) 2>&1
    if ($return.Exception)
    {
        #throw $return.Exception
        $result = 'Error'
        $errCount ++
    }
    elseif (($null -eq $_.Expect) -or ( '-' -eq $_.Expect)) {
        $_.Expect = '-'
        $result = '-'
    }
    else {
        if ($null -eq $return ) {
            $result = 'NULL'
        }
        elseif ($return -match $_.Expect) {
            $result = 'OK'
        }
        elseif ($_.Expect -is [Array]) {
            if (Compare-Object [Array]$return $_.Expect) {
                $result = 'OK'
            }
        }
        else {
            $result = 'NG'
            $ngCount ++
        }
    }
    New-Object -TypeName PSObject -Property @{
        No = $i
        Name = $_.Name
        Command = $_.command
        Expect = $_.Expect
        Return = $return
        Result = $result
    }
    $i++
} | Format-Table No, Name, Expect, Return, Result -AutoSize
Write-Output "NG:$ngCount, Error:$errCount"